In [1]:
import re
from collections import defaultdict

In [2]:
input_file = "22_input.txt"
#input_file = "22_sample.txt"

In [128]:
directions = ["right", "down", "left", "up"]

class Node:
    def __init__(self, y, x):
        self.x = x
        self.y = y
        self.neighbours = {d: None for d in directions}
        self.turn = {d: 0 for d in directions}

class Board:
    def __init__(self, map_instructions):
        self.nodes = defaultdict(dict)
        self.coords = defaultdict(list)
        self.instructions = re.split(r"([LR])", map_instructions[-1])
        mapstrings = map_instructions[:-2]
        n_col = 0
        for j, r in enumerate(mapstrings):
            n_col = max(n_col, len(r))
            for k, c in enumerate(r):
                if c == '.':
                    self.nodes[j][k] = Node(j, k)
                    self.coords[j].append(k)
                    if k in self.coords[j - 1]:
                        self.nodes[j][k].neighbours["up"] = self.nodes[j - 1][k]
                        self.nodes[j-1][k].neighbours["down"] = self.nodes[j][k]
                    if k - 1 in self.coords[j]:
                        self.nodes[j][k].neighbours["left"] = self.nodes[j][k - 1]
                        self.nodes[j][k - 1].neighbours["right"] = self.nodes[j][k]
        # add periodicity (horizontal)
        for j, r in enumerate(mapstrings):
            row = r.strip()
            if row[0] == '.' and row[-1] == '.':
                start = self.nodes[j][self.coords[j][0]]
                end = self.nodes[j][self.coords[j][-1]]
                start.neighbours["left"] = end
                end.neighbours["right"] = start
        # add periodicity (vertical):
        for k in range(n_col):
            first_i = 0
            while len(mapstrings[first_i]) <= k or mapstrings[first_i][k] == ' ':
                first_i += 1
            last_i = len(mapstrings) - 1
            while len(mapstrings[last_i]) <= k or mapstrings[last_i][k] == ' ':
                last_i -= 1
            if mapstrings[first_i][k] == '.' and mapstrings[last_i][k] == '.':
                start = self.nodes[first_i][k]
                end = self.nodes[last_i][k]
                start.neighbours["up"] = end
                end.neighbours["down"] = start
        
        # clean dicts
        self.nodes = {k: v for k, v in self.nodes.items() if v}
        self.coords = {k: v for k, v in self.coords.items() if v}
        
    def path(self):
        location = self.nodes[0][self.coords[0][0]] #first node
        facing = 0
        for i in self.instructions:
            if i == "L":
                facing = (facing - 1) % 4
            elif i == "R":
                facing = (facing + 1) % 4
            else:
                direction = directions[facing]
                for _ in range(int(i)):
                    if location.neighbours[direction] is None:
                        break
                    else:
                        location = location.neighbours[direction]
        password = 1000 * (location.y + 1) + 4 * (location.x + 1) + facing
        return password, location.y, location.x, facing


class Cube:
    def __init__(self, map_instructions):
        self.nodes = defaultdict(dict)
        self.coords = defaultdict(list)
        self.instructions = re.split(r"([LR])", map_instructions[-1])
        mapstrings = map_instructions[:-2]
        n_col = 0
        for j, r in enumerate(mapstrings):
            n_col = max(n_col, len(r))
            for k, c in enumerate(r):
                if c == '.':
                    self.nodes[j][k] = Node(j, k)
                    self.coords[j].append(k)
                    if k in self.coords[j - 1]:
                        self.nodes[j][k].neighbours["up"] = self.nodes[j - 1][k]
                        self.nodes[j-1][k].neighbours["down"] = self.nodes[j][k]
                    if k - 1 in self.coords[j]:
                        self.nodes[j][k].neighbours["left"] = self.nodes[j][k - 1]
                        self.nodes[j][k - 1].neighbours["right"] = self.nodes[j][k]
        
        # join sides
        # # 1-4
        # self.join([(0, 50 + k) for k in range(50)], [(150 + k, 0) for k in range(50)], 3, 2)
        # # 1-5
        # self.join([(k, 50) for k in range(50)], [(0, 149 - k) for k in range(50)], 2, 2)
        # # 2-4
        # self.join([(0, 100 + k) for k in range(50)], [(199, k) for k in range(50)], 3, 1)
        # # 2-6
        # self.join([(149, k) for k in range(50)], [(99, 149 - k) for k in range(50)], 0, 0)
        # # 2-3
        # self.join([(49, 100 + k) for k in range(50)], [(50 + k, 99) for k in range(50)], 1, 0)
        # # 3-5
        # self.join([(50 + k, 50) for k in range(50)], [(100, k) for k in range(50)], 2, 3)
        # # 4-6
        # self.join([(150 + k, 49) for k in range(50)], [(149, 50 + k) for k in range(50)], 0, 1)
        
        # sample
        # 1-5
        self.join([(k, 8) for k in range(4)], [(4, 4 + k) for k in range(4)], 2, 3)
        # 1-4
        self.join([], [], )
        
        
    
    
    def join(self, edge1, edge2, dir1, dir2):
        for (y1, x1), (y2, x2) in zip(edge1, edge2):
            if x1 in self.coords[y1] and x2 in self.coords[y2]:
                start = self.nodes[y1][x1]
                end = self.nodes[y2][x2]
                start.neighbours[directions[dir1]] = end
                end.neighbours[directions[dir2]] = start
                start.turn[directions[dir1]] = 2 + dir1 - dir2
                end.turn[directions[dir2]] = 2 + dir2 - dir1
        
        
    def path(self):
        location = self.nodes[0][self.coords[0][0]] #first node
        facing = 0
        for i in self.instructions:
            if i == "L":
                facing = (facing - 1) % 4
            elif i == "R":
                facing = (facing + 1) % 4
            else:
                for _ in range(int(i)):
                    direction = directions[facing]
                    if location.neighbours[direction] is None:
                        break
                    else:
                        facing = (facing + location.turn[direction]) % 4
                        location = location.neighbours[direction]
        password = 1000 * (location.y + 1) + 4 * (location.x + 1) + facing
        return password, location.y, location.x, facing


In [121]:
directions

['right', 'down', 'left', 'up']

In [122]:
with open(input_file) as f:
    inputstrings = [line.rstrip('\n') for line in f] 

In [123]:
myboard = Board(inputstrings)

In [124]:
myboard.path()

(155060, 154, 14, 0)

In [125]:
cube = Cube(inputstrings)

In [127]:
cube.path()

# <18460

(18460, 17, 114, 0)